#### Day 16, Part 2: Solving

1 am breakthrough!

In [1]:
from collections import defaultdict 
from math import factorial

filepath = "day16_data.txt"
with open(filepath) as fh:
    lines = [line for line in fh.readlines()]
    
# handle rules
rules_list = []

for _ in range(len(lines)):
    if lines[_] == '\n':
        break
    
    # build out filters ->
    num_vals = lines[_].split(": ")[1]
    cond1, cond2 = num_vals.split(' or ')
    
    # store off: 
    rules_list.append([int(v) for v in cond1.split('-')])
    rules_list.append([int(v) for v in cond2.split('-')])

# get nearby passengers
nearby_tickets = []
for _ in range(25, len(lines)):
    nearby_tickets.append([int(v) for v in lines[_].split(',')])

In [2]:
# Simply iterate over each option & run through rules: 
invalid_list = []
valid_list = [] # adding a valid list
tot_rules = len(rules_list)

for nearby_tix in nearby_tickets:
    invalid_flag = False
    for tix in nearby_tix:
        tix_count = 0
        for rule in rules_list:
            if tix < rule[0] or tix > rule[1]:
                tix_count += 1
        if tix_count >= tot_rules:
            #print(f"Tix {tix} is invalid")
            invalid_list.append(tix)
            invalid_flag = True
    # add any false invalid flags to valid list
    if not invalid_flag:
        valid_list.append(nearby_tix)
            
print(sum(invalid_list))
print(len(valid_list))

25961
190


#### Permutation Approach Won't Work: 
 - Originally planned to build every possible rule set, and then iterate through until one passed.
 - 2,432,902,008,176,640,000 total permutations....yikes. 

In [3]:
factorial(len(valid_list[0])) / factorial(0)

2.43290200817664e+18

#### New Idea: 
- We only have 190 valid users, which is pretty small. 
- Instead of handling all permutations, I can just pass in columns for all users over all rules & track how many pass. From there I can whittle down which rules connect to which. 
- This limits to the problem to be: 
    - `20 * 20 = 400 combinations of rule-column.`
    - `Each run across 190 rows each, pretty easy for Python lists`

In [4]:
# clean up rules - storing together as tuples
rules_list = list(zip(rules_list[::2], rules_list[1::2])) # package rules up properly

In [5]:
# default dict is nice for appending which rules passed
rule_dict = defaultdict(list)

# lets try over all rules - store eligible cols in dict
for i, rule in enumerate(rules_list):
    for col in range(len(valid_list[0])):
        #print(f"Testing rule {i} against column {col}")
        RulePass = True
        idx = 0
        while idx < len(valid_list):
            #print(f"On idx {idx}")
            if (valid_list[idx][col] >= rule[0][0] and valid_list[idx][col] <= rule[0][1]) or (valid_list[idx][col] >= rule[1][0] and valid_list[idx][col] <= rule[1][1]):
                idx += 1
                continue
            else:
                #print(f"Column {col} is not linked to rule {i}")
                RulePass = False
                break
        if RulePass:
            rule_dict[i].append(col)

In [6]:
# this is cool!
for key in sorted(rule_dict, key=lambda x:len(rule_dict[x]), reverse=False):
    print(key, rule_dict[key])

8 [9]
15 [8, 9]
18 [8, 9, 12]
14 [4, 8, 9, 12]
11 [4, 8, 9, 12, 18]
12 [4, 8, 9, 12, 14, 18]
16 [4, 8, 9, 11, 12, 14, 18]
5 [4, 8, 9, 11, 12, 13, 14, 18]
4 [4, 8, 9, 11, 12, 13, 14, 17, 18]
2 [1, 4, 8, 9, 11, 12, 13, 14, 17, 18]
3 [1, 4, 5, 8, 9, 11, 12, 13, 14, 17, 18]
0 [1, 4, 5, 8, 9, 11, 12, 13, 14, 16, 17, 18]
1 [1, 2, 4, 5, 8, 9, 11, 12, 13, 14, 16, 17, 18]
7 [1, 2, 4, 5, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18]
6 [1, 2, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
10 [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
19 [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
13 [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
9 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
17 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [7]:
final_dict = {}

# iterate through based on size of list; if a rule already exists, we pass
# by the end we will have a distinct dictionary that links each rule to a column
for key in sorted(rule_dict, key=lambda x:len(rule_dict[x]), reverse=False):
    rule_pass = rule_dict[key]
    for rule in rule_pass:
        if rule in final_dict.values():
            continue
        else:
            final_dict[key] = rule 

In [8]:
my_ticket = [53,101,83,151,127,131,103,61,73,71,97,89,113,67,149,163,139,59,79,137]
prod = 1

# only need to find the rules for 0-5 which represent departure
for i in range(6):
    prod *= my_ticket[final_dict[i]]

print(prod)

603409823791
